In [30]:
import pandas as pd, numpy as np, random
import json
from pathlib import Path

fixtures_root = Path("../data/processed/fixtures")
seasons = sorted([p.name for p in fixtures_root.iterdir() if p.is_dir()])

def check_fixture_calendar(season):
    cal = pd.read_csv(fixtures_root/season/"fixture_calendar.csv", parse_dates=["date_played"])
    need = {"fpl_id","fbref_id","gw_orig","date_played","team","team_id",
            "home","away","home_id","away_id","venue","gf","ga","xg","xga","poss","fdr_home","fdr_away"}
    miss = need - set(cal.columns)
    assert not miss, (season, miss)
    # venue & ids agree
    was_home = (cal["team_id"].astype(str) == cal["home_id"].astype(str)).astype("Int8")
    assert was_home.isin([0,1]).all()
    return cal

def check_player_calendar(season):
    pc = pd.read_csv(fixtures_root/season/"player_minutes_calendar.csv", parse_dates=["date_played"])
    # enforce presence of the new boolean
    assert "was_home" in pc.columns, "Add was_home in calendar_builder"
    assert pc[["player_id","date_played","venue","team_id","fbref_id"]].notna().all().all()
    # one row per player-match-team
    g = pc.groupby(["player_id","fbref_id","team_id"]).size().value_counts().to_dict()
    assert set(g) == {1}, f"Duplicates detected: {g}"
    return pc


for s in seasons:
    cal = check_fixture_calendar(s)
    pc  = check_player_calendar(s)
    # cross-check: player_minutes venue matches team ids
    ok = (pc["was_home"].astype(bool) == (pc["team_id"].astype(str) ))
    assert pc["was_home"].isin([0,1]).all(), f"{s}: was_home not in {0,1}"


print("✅ All seasons pass schema + venue checks!")


AssertionError: Duplicates detected: {}

In [ ]:

REQUIRED_COLS = {
    "player_id", "fbref_id", "team_id",
    "date_played", "venue", "was_home", "minutes"
}

def validate_player_minutes_calendar(root_dir: str, teams_lookup_path: str, players_lookup_path: str):
    root = Path(root_dir)
    seasons = sorted(p.name for p in root.iterdir() if p.is_dir())

    # Load lookup registries
    with open(teams_lookup_path, "r", encoding="utf-8") as f:
        team_lookup = json.load(f)
    with open(players_lookup_path, "r", encoding="utf-8") as f:
        player_lookup = json.load(f)

    valid_team_ids = set(str(k) for k in team_lookup.values())
    valid_player_ids = set(str(k) for k in player_lookup.values())

    for season in seasons:
        fp = root / season / "player_minutes_calendar.csv"
        print(f"Checking {season} …")
        df = pd.read_csv(fp, parse_dates=["date_played"])

        # 1. Required columns present
        missing = REQUIRED_COLS - set(df.columns)
        assert not missing, f"{season}: missing cols: {missing}"

        # 2. Coerce types for consistency
        for c in ["player_id", "fbref_id", "team_id"]:
            df[c] = df[c].astype(str)
        df["date_played"] = pd.to_datetime(df["date_played"]).dt.floor("D")
        df["was_home"] = df["was_home"].astype("Int8")

        # 3. No nulls in key columns
        key_cols = ["player_id","fbref_id","team_id","date_played","was_home"]
        assert df[key_cols].notna().all().all(), f"{season}: nulls in join keys"

        # 4. was_home matches IDs
        #pred = (df["team_id"] == df["home_id"]).astype("Int8")
        #mismatch = (df["was_home"] != pred)
        #assert not mismatch.any(), f"{season}: was_home mismatch | sample:\n{df.loc[mismatch].head()}"

        # 5. One row per player-match-team
        dup_counts = df.groupby(["player_id","fbref_id","team_id"]).size().value_counts().to_dict()
        assert set(dup_counts) == {1}, f"{season}: duplicates {dup_counts}"

        # 6. Team IDs exist in lookup
        bad_teams = set(df["team_id"]) - valid_team_ids
        assert not bad_teams, f"{season}: team_ids not in lookup: {bad_teams}"

        # 7. Player IDs exist in lookup
        bad_players = set(df["player_id"]) - valid_player_ids
        assert not bad_players, f"{season}: player_ids not in lookup: {bad_players}"

    print("✅ All seasons pass schema, type, null, venue, uniqueness, and ID registry checks!")

# Example usage:
validate_player_minutes_calendar(
    "../data/processed/fixtures",
    "../data/processed/_id_lookup_teams.json",
    "../data/processed/_id_lookup_players.json"
)


Checking 2020-2021 …
Checking 2021-2022 …
Checking 2022-2023 …
Checking 2023-2024 …
Checking 2024-2025 …
✅ All seasons pass schema, type, null, venue, uniqueness, and ID registry checks!


In [53]:
import pandas as pd, pathlib as p

season = "2023-2024"  # change
fp = p.Path("../data/processed/fixtures")/season/"player_minutes_calendar.csv"

df   = pd.read_csv(fp, parse_dates=["date_played"])
miss = df[df["price"].isna()].copy()

print(f"{season}: {len(miss)} rows with missing price")
print(miss[["player_id","player", "team","date_played","fbref_id","gw_orig","minutes","was_home"]]
      .sort_values(["date_played","team","player"])
      .head(289).to_string(index=False))


2023-2024: 76 rows with missing price
player_id          player team date_played fbref_id  gw_orig  minutes  was_home
 63e990d0         Vitinho  BUR  2023-08-11 3a6836b4        1       90         1
 63e990d0         Vitinho  BUR  2023-09-30 a79ff136        7        3         0
 63e990d0         Vitinho  BUR  2023-10-07 cfd83ca3        8       60         1
 4f7e9a8f Yehor Yarmoliuk  BRE  2023-10-21 52781f37        9       15         1
 4f7e9a8f Yehor Yarmoliuk  BRE  2023-10-28 91e3b922       10       15         0
 63e990d0         Vitinho  BUR  2023-10-28 c4c42d3e       10       90         0
 4f7e9a8f Yehor Yarmoliuk  BRE  2023-11-04 3235dd6e       11       12         1
 63e990d0         Vitinho  BUR  2023-11-04 8797f9a9       11       90         1
 63e990d0         Vitinho  BUR  2023-11-11 2cb4e4dc       12       87         0
 63e990d0         Vitinho  BUR  2023-11-25 36a64522       13       90         1
 7ea048b6   Amadou Diallo  NEW  2023-11-25 8c8f48f4       13        1         1
 6